In [ ]:
# Relating number of COVID-19 deaths and cases
> Death rates, Infection-spread rates &  Infections Per million Popolation

- comments: true
- author: Guneet Singh
- categories: [rate, compare, growth]
- hide: false
- image: images/deth_rate_comparision.png
- permalink: /covid-compare-death-rate/

In [1]:
#hide
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
%config InlineBackend.figure_format = 'retina'
import load_covid_data 
import utils
chart_width = 550
chart_height= 400
atLeast= 1000
moreThan= 50000
data = utils.getComulativeData('confirmed')
ddata = utils.getComulativeData('deaths')


In [2]:
#hide
def death_rate(totalDeath,totalCases):
    if totalCases!=0 :
        return (each['total_deaths'][i]/each['total_confirmed'][i])*100 
    else: return 0

country_data=[]
for i in data.location.unique():
    filter_country = data["location"]== i
    temp=data[filter_country]
    temp['total_deaths']=ddata[filter_country]['total_deaths']
    temp=temp.reset_index()
    temp=temp.drop('index',axis=1)
    country_data.append(temp)

every_country_data=pd.DataFrame(columns={'location', 'date', 'total_confirmed','total_deaths','percent_change','death_percent_change','death_rate'})
for each in country_data :
    each['percent_change']=None
    each['death_percent_change']=None
    each['death_rate']=None
    for i in range(each.total_confirmed.size):
         each['percent_change'][i]=utils.toPercent(i,each['total_confirmed'][i],each['total_confirmed'])
         each['death_percent_change'][i]=utils.toPercent(i,each['total_deaths'][i],each['total_deaths'])
         each['death_rate'][i]=death_rate(each['total_confirmed'][i],each['total_deaths'][i])
    # display(each)
    every_country_data=every_country_data.append(each)

temp_rank_data=pd.DataFrame(columns=['country','total'])#,data={'country':data.location.unique(),'total':None})
for each in country_data:
        # if each.death_rate.tail(1).to_list()[0]>0:
    if each.total_confirmed.tail(1).to_list()[0]>moreThan:
        temp_rank_data=temp_rank_data.append([{'country':each.location[0],'total':each.death_rate.tail(1).to_list()[0]}])
temp_rank_data=temp_rank_data.sort_values(by='total',ascending=False)


In [3]:
#hide
countries=temp_rank_data.country[0:20]
country_data=[]
for i in countries:
    filter_country = data["location"]== i
    temp=data[filter_country]
    temp['total_deaths']=ddata[filter_country]['total_deaths']
    temp=temp.reset_index()
    temp=temp.drop('index',axis=1)
    country_data.append(temp)

every_country_data=pd.DataFrame(columns={'location', 'date', 'total_confirmed','total_deaths','percent_change','death_percent_change','death_rate'})
for each in country_data :
    each['percent_change']=None
    each['death_percent_change']=None
    each['death_rate']=None
    for i in range(each.total_confirmed.size):
         each['percent_change'][i]=utils.toPercent(i,each['total_confirmed'][i],each['total_confirmed'])
         each['death_percent_change'][i]=utils.toPercent(i,each['total_deaths'][i],each['total_deaths'])
         each['death_rate'][i]=death_rate(each['total_confirmed'][i],each['total_deaths'][i])
    # display(each)
    every_country_data=every_country_data.append(each)
# pd.set_option('display.max_rows', None)
# every_country_data[every_country_data.location=='Iran']

# Death Rate

## Countries with >50000 cases (Projected starting 1000 case mark to calculate rise of deaths in terms of percentage).

In [27]:
#hide_input
source= every_country_data[every_country_data.total_confirmed>atLeast]
# display(source)

scales = alt.selection_interval(bind='scales', zoom=True)
selection = alt.selection_multi(fields=['location'], bind='legend')
alt.data_transformers.disable_max_rows()

base = alt.Chart(source, title = "Death rate").encode(
    x = alt.X('date:T', title = "Days passed"),
    y = alt.Y("death_rate:Q",title = "Percentage"),
    color = alt.Color('location:N', legend=alt.Legend(title="Country",values=countries.to_list(), labelFontSize=15, titleFontSize=17),
                     scale=alt.Scale(scheme='tableau20')),
    opacity = alt.condition(selection, alt.value(1), alt.value(0.1))
)

lines = base.mark_line().add_selection(
    scales
).add_selection(
    selection
).properties(
    width=chart_width,
    height=chart_height
)

lines = base.mark_line().add_selection(
    scales
).add_selection(
    selection
).properties(
    width=chart_width,
    height=chart_height
)
lines

alt.Chart(...)

In [29]:
#hide-input
source=every_country_data[every_country_data.total_confirmed>atLeast]

death_table=pd.DataFrame(columns={'Country','Death rate'})
for each in countries:
    temp=source[source['location']==each]
    # display(temp)
    death_table=death_table.append([{'Country':each, 'Death rate' : temp.death_rate.tail(1).to_list()[0]}])
death_table=death_table.reset_index().drop('index',axis=1)
death_table

,Death rate,Country
0,16.399573,Belgium
1,11.902501,Spain
2,14.060315,Italy
3,14.371694,United Kingdom
4,15.323419,France
5,6.058653,United States
6,7.477835,Canada
7,4.518621,Germany
8,5.984301,Iran
9,2.812516,Peru


# Infection-spread rates

## Countries with >50000 cases (Projected starting 1000 case mark to calculate rise of infection in terms of percentage)

In [8]:
#hide_input
source= every_country_data[every_country_data.total_confirmed>atLeast]

scales = alt.selection_interval(bind='scales', zoom=True)
selection = alt.selection_multi(fields=['location'], bind='legend')
alt.data_transformers.disable_max_rows()

base = alt.Chart(source, title = "Infection-spread rate").encode(
    x = alt.X('date:T', title = "Days passed"),
    y = alt.Y("percent_change:Q",title = "Percentage"),
    color = alt.Color('location:N', legend=alt.Legend(title="Country",values=countries.to_list(), labelFontSize=15, titleFontSize=17),
                     scale=alt.Scale(scheme='tableau20')),
    opacity = alt.condition(selection, alt.value(1), alt.value(0.1))
)

lines = base.mark_line().add_selection(
    scales
).add_selection(
    selection
).properties(
    width=chart_width,
    height=chart_height
)

lines = base.mark_line().add_selection(
    scales
).add_selection(
    selection
).properties(
    width=chart_width,
    height=chart_height
)
lines


alt.Chart(...)

In [30]:
#hide_input
source= every_country_data[every_country_data.total_confirmed>atLeast]

death_table=pd.DataFrame(columns={'Country','Spread rate'})
for each in countries:
    temp=source[source['location']==each]
    # display(temp)
    death_table=death_table.append([{'Country':each, 'Spread rate' : temp.percent_change.tail(1).to_list()[0]}])
death_table=death_table.reset_index().drop('index',axis=1)
death_table

,Country,Spread rate
0,Belgium,0.568719
1,Spain,0.371243
2,Italy,0.446638
3,United Kingdom,1.495768
4,France,0.454586
5,United States,1.968346
6,Canada,1.648815
7,Germany,0.218268
8,Iran,1.603903
9,Peru,5.632585


In [9]:
#hide
ddata = ddata.rename(columns={"total_deaths": "total_cases"})
ddata.columns

# Population data
data_pop=pd.read_csv("data_files/world_population.csv")[["country","population"]]
data_pop=data_pop[data_pop.country.isin(countries)]
# To list the countries who's population is not found in world bank
# pd.set_option('display.max_rows', None)
# countries=countries[~countries.isin(data_pop.country)]
countries=countries[countries.isin(data_pop.country)]


# display(data_pop.country.unique().size , countries.size)
# (countries.isin(data_pop.country))[countries == False]
data_pc=ddata.copy()
data_pc=data_pc[data_pc.location.isin(countries)]

In [10]:
#hide
for i in countries:
    data_pc.loc[data_pc.location == i,"total_cases"] = data_pc.loc[data_pc.location == i,"total_cases"]/(data_pop.loc[data_pop.country == i, "population"].astype(float).to_list()[0]/1000000)

data_countries_pc=[]

ranked_data=pd.DataFrame(columns=['country','total_cases_per_milliion_pop'])#country, total_cases_per_milliion_pop
# ranked_data.append({'a','b'})
for i in countries:
    filter_country = data_pc["location"]== i
    temp=data_pc[filter_country]
    # display(temp)
    # if temp['total_cases'].tail(1).to_list()[0]> 500:
    if temp['total_cases'].sum() > 0 :
        ranked_data=ranked_data.append([{'country':i,'total_cases_per_milliion_pop':temp['total_cases'].tail(1).to_list()[0]}])

ranked_data=ranked_data.sort_values(by='total_cases_per_milliion_pop',ignore_index=True,ascending=False)
countries=ranked_data['country']
countries=countries[0:20]
for i in countries:
    filter_country = data_pc["location"]== i
    # temp=data_pc[filter_country]
    data_countries_pc.append(data_pc[filter_country])
# pd.set_option('display.max_rows', None)
# display(ranked_data)
for i in range(0,len(countries)):
    data_countries_pc[i]=data_countries_pc[i].reset_index()
    data_countries_pc[i]['x_day'] = data_countries_pc[i].index
len(data_countries_pc)

data_plot=pd.DataFrame()
for i in range(0,len(countries)):
    data_plot = pd.concat([data_plot, data_countries_pc[i]], axis=0)
# data_plot

data_plot_death_log=data_plot.copy()
def a(x):
    if x==0:
        return 0
    return np.log(x)
pd.set_option('display.max_rows', 100)

# data_plot['total_log_cases']=data_plot['total_cases'].apply(a)
data_plotdata_plot=pd.DataFrame()
for i in range(0,len(countries)):
    data_plot = pd.concat([data_plot, data_countries_pc[i]], axis=0)
# data_plot

data_plot_death_log=data_plot.copy()
def a(x):
    if x==0:
        return 0
    return np.log(x)
pd.set_option('display.max_rows', 100)

# data_plot['total_log_cases']=data_plot['total_cases'].apply(a)
# data_plot

# Deaths Per Million Population

In [11]:
#hide_input
# Plot it using Altair
source = data_plot

scales = alt.selection_interval(bind='scales', zoom=True)
selection = alt.selection_multi(fields=['location'], bind='legend')
alt.data_transformers.disable_max_rows()

base = alt.Chart(source, title = "20 Highest COVID-19 Deaths Per Million of Inhabitants").encode(
    x = alt.X('date:T', title = "Days passed"),
    y = alt.Y("total_cases:Q",title = "Deaths per million"),
    color = alt.Color('location:N', legend=alt.Legend(title="Country",values=countries.to_list(), labelFontSize=15, titleFontSize=17),
                     scale=alt.Scale(scheme='tableau20')),
    opacity = alt.condition(selection, alt.value(1), alt.value(0.1))
)

lines = base.mark_line().add_selection(
    scales
).add_selection(
    selection
).properties(
    width=chart_width,
    height=chart_height
)

lines = base.mark_line().add_selection(
    scales
).add_selection(
    selection
).properties(
    width=chart_width,
    height=chart_height
)
lines

alt.Chart(...)

In [35]:
#hide_input

source=data_plot

death_table=pd.DataFrame(columns={'Country','Deaths/ml','Total Population'})
for each in countries:
    temp=source[source['location']==each]
    # display(temp)
    death_table=death_table.append([{'Country':each, 'Deaths/ml' : temp.total_cases.tail(1).to_list()[0], 'Total Population':data_pop.loc[data_pop.country == each, "population"].astype(int).to_list()[0]}])
death_table=death_table.reset_index().drop('index',axis=1)
death_table

,Country,Deaths/ml,Total Population
0,Belgium,768.187196,11589623
1,Spain,584.346695,46754778
2,Italy,518.806693,60461826
3,United Kingdom,496.317275,67886011
4,France,420.201083,65273511
5,United States,259.508496,331002651
6,Canada,148.163245,37742154
7,Germany,94.099177,83783942
8,Iran,81.602088,83992949
9,Peru,68.755612,32971854


>The data suggests that with increasing number of cases increases death rate up to a certain point but also slows down the spread(rate) of the virus.

Updated daily by [GitHub Actions](https://github.com/features/actions).

This visualization was made by [Guneet Singh](https://github.com/guneetgstar).

- Data sourced from [World Bank](https://data.worldbank.org/indicator/SP.POP.TOTL) ["2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/), [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [original notebook](https://github.com/guneetgstar/covid19/blob/master/_notebooks/2020-05-15-relation-in-deaths-and-number-of-cases.ipynb).